In [1]:
pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
      --------------------------------------- 0.0/3.0 MB 653.6 kB/s eta 0:00:05
     --- ------------------------------------ 0.3/3.0 MB 2.6 MB/s eta 0:00:02
     --------------- ------------------------ 1.1/3.0 MB 7.9 MB/s eta 0:00:01
     ---------------------------------------  2.9/3.0 MB 17.1 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 14.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/73.5 kB ? eta -:--:--
   ---------------------------------------- 73.5/73.5 kB 4.0 MB/

In [2]:
pip install yfinance pandas requests

Note: you may need to restart the kernel to use updated packages.


In [27]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Function to fetch the list of S&P 500 companies from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip().replace('.', '-')
        tickers.append(ticker)
    return tickers

# Fetch the list of S&P 500 tickers
sp500_tickers = get_sp500_tickers()

# Function to fetch stock data for a list of tickers
def fetch_stock_data(tickers, start_date, end_date):
    all_data = []
    problematic_tickers = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(start=start_date, end=end_date)
            if not hist.empty:
                hist.reset_index(inplace=True)
                hist['Ticker'] = ticker
                all_data.append(hist)
            else:
                print(f"No data found for {ticker}")
                problematic_tickers.append(ticker)
        except Exception as e:
            print(f"Could not download data for {ticker}: {e}")
            problematic_tickers.append(ticker)
    if all_data:
        return pd.concat(all_data, ignore_index=True), problematic_tickers
    else:
        return pd.DataFrame(), problematic_tickers

# Fetch data for the desired date range
start_date = '2023-01-01'
end_date = '2023-12-31'
stock_data, problematic_tickers = fetch_stock_data(sp500_tickers, start_date, end_date)

if not stock_data.empty:
    # Format the DataFrame
    stock_data = stock_data[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    stock_data['Adj Close'] = stock_data['Close']  # Assuming 'Adj Close' is the same as 'Close' for simplicity
    stock_data['Adj Volume'] = stock_data['Volume']  # Assuming 'Adj Volume' is the same as 'Volume' for simplicity

    # Save to CSV
    file_path = 'sp500_stock_prices.csv'
    stock_data.to_csv(file_path, index=False)

    print("CSV file saved at:", file_path)
else:
    print("No stock data available to save.")

if problematic_tickers:
    print("Problematic tickers:", problematic_tickers)


GEV: Data doesn't exist for startDate = 1672549200, endDate = 1703998800


No data found for GEV


SW: Data doesn't exist for startDate = 1672549200, endDate = 1703998800


No data found for SW


SOLV: Data doesn't exist for startDate = 1672549200, endDate = 1703998800


No data found for SOLV
CSV file saved at: sp500_stock_prices.csv
Problematic tickers: ['GEV', 'SW', 'SOLV']
